In [1]:
NAME = "Nick Morton"
COLLABORATORS = "Quan Tran"

In [31]:
from requests_oauthlib import OAuth1Session
import requests
from requests_oauthlib import OAuth1
import json
import pandas as pd
import re
import sqlalchemy as sa

---

# Twitter Data Acquisition Notebook

## 1. Authorization & Authentication

### Obtaining the access key and secret from the token file.

In [120]:
def getAccessTokens(token_file_name):
    token_file = open(token_file_name, "r")

    cred_str = token_file.read()
    cred = json.loads(cred_str)
    token_file.close()
    
    return cred['key'], cred['secret'], cred['client_key'], cred['client_secret']

In [121]:
access_token_key, access_token_secret, client_key, client_secret = getAccessTokens('tokens.txt')

### Creating an OAuth object to be reused

In [122]:
oauth = OAuth1(client_key,
                   client_secret=client_secret,
                   resource_owner_key=access_token_key,
                   resource_owner_secret=access_token_secret)

## 2. Retrieving tweets from public Twitter profiles. 

- We first write a function `getTweetDf` to retrieve the tweets from the Twitter profile with the given name, and put the major fields (id, username, tweet text, favorite count and retweet count) into a DataFrame and return it.

In [10]:
def getTweetDf(screen_name, oauth):
    # Specify the API endpoint and parameters
    protected_url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
    params = {'screen_name': screen_name, 'count': 200, 'exclude_replies': 'true', 'include_rts': 'false'}
    tweets = []
    
    # We only need a maximum of 900 tweets per Twitter account
    while len(tweets) < 900:
        r = requests.get(url=protected_url, auth=oauth, params=params)
        r = r.json()
        if len(r) == 0:
            break
            
        lastTweet = r[-1]
        params['max_id'] = lastTweet['id'] - 1
        
        tweets.extend(r)
        
    # Put 900 tweets into a DataFrame and keep the important fields
    tweets = tweets[:900]
    tweetsDf = pd.DataFrame(tweets)
    tweetsDf = tweetsDf[['id', 'user', 'text', 'favorite_count', 'retweet_count']]
    
    # Use the screen name for the user column instead of the whole user object
    for index, row in tweetsDf.iterrows():
        tweetsDf.at[index, 'user'] = tweetsDf.loc[index, 'user']['screen_name']
    
    return tweetsDf

- Next, we have the function `getWordFreqsDf` which takes a DataFrame of tweets, looks at their texts and computes the frequency of each word that appears in the tweets.
- The function also computes the average numbers of favorites and retweets of the tweets that each word appears in.

In [44]:
def getWordFreqsDf(tweetDf, groupNumber):
    wordFreq = {}

    for index, row in tweetDf.iterrows():
        words = re.split('\W+', row['text'])
        
        # Compute the frequency of each word in a single tweet
        wordFreqInTweet = {}
        for word in words:
            if word in wordFreqInTweet:
                wordFreqInTweet[word] += 1
            else:
                wordFreqInTweet[word] = 1
        
        # For each word, we store its frequency, the number of tweets
        # it appears in, and the total numbers of favorites and retweets.
        for word in wordFreqInTweet:
            if word in wordFreq:
                wordFreq[word]['freq'] += wordFreqInTweet[word]
                wordFreq[word]['tweets_count'] += 1
                wordFreq[word]['retweet'] += row['retweet_count']
                wordFreq[word]['favorite'] += row['favorite_count']
            else:
                wordFreq[word] = {'freq': 1, 'tweets_count': 1, 
                                  'retweet': row['retweet_count'], 
                                  'favorite': row['favorite_count'],
                                  'avg_retweet': 0, 'avg_favorite': 0,
                                  'group': groupNumber}
    
    # Compute the average numbers of retweets and favorites.
    for word in wordFreq:
        wordFreq[word]['avg_retweet'] = wordFreq[word]['retweet'] / wordFreq[word]['tweets_count']
        wordFreq[word]['avg_favorite'] = wordFreq[word]['favorite'] / wordFreq[word]['tweets_count']
    
    # Put all the data in a table and convert it to a DataFrame
    wordFreqTable = []
    wordFreqTable.append(['Word', 'Frequency', 'Average Retweets', 'Average Favorites', 'Group'])
    for word in wordFreq:
        wordFreqTable.append([word, wordFreq[word]['freq'], 
                              wordFreq[word]['avg_retweet'], 
                              wordFreq[word]['avg_favorite'],
                              wordFreq[word]['group']])
        
    df = pd.DataFrame(wordFreqTable)
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    
    return df

- Next, we get the tweets from the athletes' Twitter accounts and compute the word frequencies.

In [25]:
# Retrieve the tweets
lbj = getTweetDf('KingJames', oauth)
neymar = getTweetDf('neymarjr', oauth)
kaka = getTweetDf('KAKA', oauth)
athletes = [lbj, neymar, kaka]
athletes = pd.concat(athletes) 

print(len(athletes))
athletes.head()

2700


,id,user,text,favorite_count,retweet_count
0,985157276750237697,KingJames,Preach “PrimeTime”! Once u understand that you...,22448,4098
1,983544585086529537,KingJames,All love and respect Young King 🤴🏾! Good luck ...,52696,5635
2,983341526930554880,KingJames,Just heard a new @jerreau track! SHEESH!! “Y’a...,8485,852
3,982021236002820096,KingJames,Can’t wait to go see it! Wife and I be waiting...,15957,1330
4,981698022269685760,KingJames,🔥🔥🔥🔥🔥🔥🔥🔥. https://t.co/hdP1K0mSk5,33623,4620


In [45]:
# Compute the word frequencies
athletesWordFreq = getWordFreqsDf(athletes, 1)
athletesWordFreq.to_csv('athletes.csv', index = False)

athletesWordFreq.head()

,Word,Frequency,Average Retweets,Average Favorites,Group
1,Preach,1,4098,22448,1
2,PrimeTime,1,4098,22448,1
3,Once,5,9867.8,35390.6,1
4,u,74,7005.26,20022.6,1
5,understand,2,29455.5,83218,1


- We get the tweets from the actors' Twitter accounts and compute the word frequencies.

In [14]:
# Retrieve the tweets
gomez = getTweetDf('selenagomez', oauth)
hart = getTweetDf('KevinHart4real', oauth)
dicaprio = getTweetDf('LeoDiCaprio', oauth)
actors = [gomez, hart, dicaprio]
actors = pd.concat(actors)

print(len(actors))
actors.head()

2540


,id,user,text,favorite_count,retweet_count
0,992424215167959041,selenagomez,I wanna hold you when I’m not supposed to… htt...,152101,26475
1,992071860140392448,selenagomez,Are you guys ready for @13ReasonsWhy Season 2 ...,39460,10323
2,991708813404209152,selenagomez,Presave #BackToYou and you’ll be entered to wi...,44315,9984
3,991347554984935424,selenagomez,"#BackToYou. Out May 10th, part of the @13Reaso...",93109,27717
4,987759479419621377,selenagomez,So inspired by your work ♥️ https://t.co/bkQdR...,43124,7192


In [48]:
# Compute the word frequencies
actorsWordFreq = getWordFreqsDf(actors, 2)
actorsWordFreq.to_csv('actors.csv', index = False)

actorsWordFreq.head()

,Word,Frequency,Average Retweets,Average Favorites,Group
1,I,670,6038.37,13728.6,2
2,wanna,8,13344.9,37794.9,2
3,hold,7,13061.6,44720.4,2
4,you,382,9142.83,19907.3,2
5,when,23,8942.3,23393.3,2


- Finally, we get the tweets from the students' Twitter accounts and compute the word frequecies.

In [17]:
student1 = getTweetDf('the_nick_oftime', oauth)
student2 = getTweetDf('monica_woolcott', oauth)
student3 = getTweetDf('joshiscoolblaz', oauth) 
student4 = getTweetDf('Alex_Mintonn', oauth)
student5 = getTweetDf('man_to_follow56', oauth)
student6 = getTweetDf('Emilie21Fisher', oauth)
student7 = getTweetDf('jswickard2', oauth) 
student8 = getTweetDf('SStrittmatter9', oauth)
student9 = getTweetDf('mcgagh_john', oauth)
student10 = getTweetDf('nkeirn16', oauth)
student11 = getTweetDf('mrcarter_2', oauth)
student12 = getTweetDf('smccauley_12', oauth)
student13 = getTweetDf('Cgqb7', oauth)
student14 = getTweetDf('BenHarder65', oauth)
student15 = getTweetDf('morganhattenn', oauth)
student16 = getTweetDf('jack___savage', oauth)
student17 = getTweetDf('jarodscheff15', oauth)
student18 = getTweetDf('ZellsBells74_UH', oauth)
student19 = getTweetDf('OhhhhGawd', oauth)
student20 = getTweetDf('JEltringham', oauth) 
student21 = getTweetDf('NoahRichards1', oauth)
student22 = getTweetDf('kosiorCatherine', oauth)
student24 = getTweetDf('JakePedelose', oauth)

students = [student1, student2, student3, student4, student5, student6, student7, student8, student9, student10, 
           student11, student12, student13, student14, student15, student16, student17, student18, student19, student20, 
           student21, student22, student24]
students = pd.concat(students)
students.head()

,id,user,text,favorite_count,retweet_count
0,993363452365242369,the_nick_oftime,Favorite thing on Twitter aside @stoolpresiden...,0,0
1,989705403511529472,the_nick_oftime,Drafted a senior citizen to play tight end,8,1
2,960368592842903552,the_nick_oftime,#ThisIsUs https://t.co/zQRT1LMMTY,4,1
3,952039120053919744,the_nick_oftime,Poor mans drops his pizza but how does he get ...,12,0
4,950595394353946624,the_nick_oftime,Dilly dilly. Hotrod. Dilly dilly. https://t.co...,1,0


In [47]:
studentsWordFreq = getWordFreqsDf(students, 3)
studentsWordFreq.to_csv('students.csv', index = False)

studentsWordFreq.head()

,Word,Frequency,Average Retweets,Average Favorites,Group
1,Favorite,5,0,1.8,3
2,thing,136,0.669118,4.71324,3
3,on,900,0.520737,4.46198,3
4,Twitter,32,0.322581,3.96774,3
5,aside,1,0,0,3


--------

## 3. Retrieving tweets from the Democratic Party's Twitter accounts.

- Since we are given the Tweet ID of each tweets, we write a `getTweet` function that retrieve a Tweet using its ID

In [36]:
def getTweet(tweetId, oauth):
    protected_url = 'https://api.twitter.com/1.1/statuses/show.json'
    params = {'id': tweetId}

    r = requests.get(url=protected_url, auth=oauth, params=params)
    tweet = r.json()
    try:
        tweet['id'] = str(tweet['id'])
        tweet['text'] = str(tweet['text'])
    except:
        # return None if failed to retrieve the tweet
        return None
        
    return tweet


- Next, we have a function that open a file containing the tweet ID's and retrieve the specified number of tweets from Twitter using the Twitter API.

In [116]:
def getTweetsFromFile(filename, oauth, tweetsCount, offset):
    tweetIdsFile = open(filename, 'r')
    counter = 0
    total_calls = 0
    lineCounter = 0
    tweets = []

    for line in tweetIdsFile:
        # There is a rate limit of 900 calls/15-minute period for
        # the API endpoint, therefore we keep track of the total 
        # number of calls we've made so far
        if total_calls >= 900:
            break
            
        # offset is used to specify how far into the file do we start 
        # retrieving tweets from Twitter
        lineCounter += 1
        if lineCounter <= offset:
            continue
            
        # Keep track of how many tweets we've stored.
        if counter >= tweetsCount:
            break
            
        # Retrieve the tweet ID and the tweet.
        tweetId = line.strip()
        tweet = getTweet(tweetId, oauth)
        total_calls += 1
        
        # Omit failed results or retweeted tweets
        if tweet == None or 'retweeted_status' in tweet:
            continue
        else:
            counter += 1
            tweets.append(tweet)

    tweetIdsFile.close()

    # Put the tweets into a DataFrame
    tweetsDf = pd.DataFrame(tweets)
    tweetsDf = tweetsDf[['id', 'user', 'text', 'favorite_count', 'retweet_count']]

    # Properly format the usernames and the tweet ID's.
    for index, row in tweetsDf.iterrows():
        tweetsDf.at[index, 'user'] = tweetsDf.loc[index, 'user']['screen_name']
        tweetsDf.at[index, 'id'] = str(tweetsDf.loc[index, 'id'])
        
    return tweetsDf

In [119]:
# Creating an empty DataFrame
offset = 0
demPartyTweets = {'id': [], 'user': [], 'text': [], 'favorite_count': [], 'retweet_count': []}
demPartyTweets = pd.DataFrame(demPartyTweets)

- Because there is a rate limit on how many calls to the API endpoint we can make (900 calls/15-minute window), we have to put the calls to the API in a while block and keep trying until we get enough data, because it's not possible to get all 2700 tweets at once.

- Note that the following code block will have to run for approximately 45 minutes.

In [123]:
# we only want around 2700 tweets
while offset < 2700:
    tmpDf = getTweetsFromFile('democratic-party-timelines.txt', oauth, 100, offset)
    demPartyTweets = [demPartyTweets, tmpDf]
    demPartyTweets = pd.concat(demPartyTweets)

    offset = len(demPartyTweets)

demPartyTweets.head()

,favorite_count,id,retweet_count,text,user
0,295.0,786577430295306240,251.0,No. https://t.co/2QcAQhrGae,TheDemocrats
1,1785.0,786642187912114176,972.0,Trump doesn't respect women. Or minorities. Or...,TheDemocrats
2,241.0,796001834473095170,164.0,RT if you're ready to elect some Democrats. ht...,TheDemocrats
3,260.0,795996805670445057,242.0,"RT if you've gotten your ""I Voted"" sticker, he...",TheDemocrats
4,649.0,778362374290018304,564.0,Trump used more than a quarter-million dollars...,TheDemocrats


In [46]:
demWordFreqs = getWordFreqsDf(demPartyTweets, 4)
demWordFreqs.head()

,Word,Frequency,Average Retweets,Average Favorites,Group
1,No,6,132,181.25,4
2,https,380,117.87,166.194,4
3,t,411,119.82,170.154,4
4,co,380,117.87,166.194,4
5,2QcAQhrGae,1,251,295,4


## 4. Putting everything into a SQL Database.

- Now that we have all the data we need, we create a SQL database and start putting data in.

- First we use the `getCreds` function to obtain the credentials to connecting to the SQL Database on Hadoop2.

In [26]:
def getCreds(filename, subset, defaults = {}):
    """ Use `filename` to look for a file containing a json-encoded dictionary
        of credentials.  If the file is successfully found and contains valid
        json, return the sub-dictionary based on `subset`.  If the file is not
        found, is not accessible, has improper encoding, or if the subset is 
        not present in the dictionary, return the given defaults.
    """
    try:
        with open(filename, 'r') as file:
            D = json.load(file)
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

creds = getCreds("mysql_creds.json", "mysql", defaults={'user': 'studen_j1', 
                                                  'password': 'studen_j1'})

- Next we use the function `db_setup` to create a connection and its engine:

In [62]:
def db_setup(user, password, database):
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(user, password, database)
    e = sa.create_engine(cstring)
    c = e.connect()
    return e, c

In [68]:
try:
    connection.close()
    del engine
except:
    pass

engine, connection = db_setup(creds['user'], creds['password'], creds['user'])
database = creds['user']

- We then proceed to creating 2 tables to store the data.

- The first table is used to stored all the tweets we retrieved, along with their texts, id's, favorite count, retweet count and the usernames of the tweets' owners.

id | username | text | favorite | retweet
---|----------|------|----------|----------
1  | quantran312 | Sample tweet | 12 | 3

- The second table is used to stored the information of all the words that appears in the tweets: the word, its frequency, average favorite countand retweet count, and the group (of the 4 groups) that the word belongs to.

word | frequency | avg_favorite | avg_retweet | group
-----|-----------|--------------|-------------|-------
thank| 2313      | 1235.1231    | 634.512     | 1 

In [71]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [65]:
%sql $cstring

'Connected: tran_q1@'

In [69]:
%sql USE $database

0 rows affected.


[]

In [101]:
%%sql

CREATE TABLE TWEETS(
    ID VARCHAR(128)       NOT NULL,
    USERNAME VARCHAR(256) NOT NULL,
    TEXT TEXT             NOT NULL,
    FAVORITE INT          NOT NULL,
    RETWEET INT           NOT NULL,
    PRIMARY KEY (ID)
);

0 rows affected.


[]

In [110]:
%%sql

CREATE TABLE WORDS(
    WORD VARCHAR(256)             NOT NULL,
    FREQUENCY INT                 NOT NULL,
    AVG_FAVORITE DECIMAL(18, 3)   NOT NULL,
    AVG_RETWEET DECIMAL(18, 3)    NOT NULL,
    GROUP_NUM INT                 NOT NULL
);

0 rows affected.


[]

- Next we go through all 4 DataFrames that contain the tweets for the 4 groups, and put them into the MySQL database.

In [90]:
def insertTweets(connection, tweetsDf):
    insertQuery = sa.sql.text("INSERT INTO TWEETS VALUES \
                              (:id, :username, :text, :favorite, :retweet)")
    
    for index, tweet in tweetsDf.iterrows():
        boundInsertQuery = insertQuery.bindparams(id=tweet['id'],
                                                 username=tweet['user'],
                                                 text=tweet['text'],
                                                 favorite=tweet['favorite_count'],
                                                 retweet=tweet['retweet_count'])
        connection.execute(boundInsertQuery)

In [98]:
insertTweets(connection, athletes)
insertTweets(connection, actors)
insertTweets(connection, students)
insertTweets(connection, demPartyTweets)

- Lastly we go through all the 4 DataFrames that contain the word frequencies for the 4 groups and put them into the database.

In [106]:
def insertWordFreqs(connection, wordFreqsDf):
    insertQuery = sa.sql.text("INSERT INTO WORDS VALUES\
                              (:word, :freq, :avg_favorite, :avg_retweet, :group_num)")
    
    for index, row in wordFreqsDf.iterrows():
        boundInsertQuery = insertQuery.bindparams(word=row['Word'],
                                                  freq=row['Frequency'],
                                                  avg_favorite=row['Average Favorites'],
                                                  avg_retweet=row['Average Retweets'],
                                                  group_num=row['Group'])
        connection.execute(boundInsertQuery)

In [111]:
insertWordFreqs(connection, athletesWordFreq)
insertWordFreqs(connection, actorsWordFreq)
insertWordFreqs(connection, studentsWordFreq)
insertWordFreqs(connection, demWordFreqs)